In [65]:
import requests
import json
import pandas as pd
import xlwings as xw
from datetime import datetime

In [66]:
auswahl = input("DAX = 0; STOXX = 1   :")
if auswahl == "1":
# STOXX
    url = 'https://www.eurex.com/api/v1/overallstatistics/69660'
    print("STOXX")
else:
# DAX
    url = 'https://www.eurex.com/api/v1/overallstatistics/70044'
    print ("DAX")

DAX = 0; STOXX = 1   :0
DAX


In [67]:
def clear():
 
    # for windows
    if name == 'nt':
        _ = system('cls')
 
    # for mac and linux(here, os.name is 'posix')
    else:
        _ = system('clear')



In [68]:
def get_idx(list_idxs):
    while(True):
        date_idxs = [0, 1]
        #date_idxs = input("Numbers corresponding to the desired dates separated by spaces: ").split(' ')
        date_idxs = list(map(int,date_idxs))
        date_idxs = [idx for idx in date_idxs if idx in list_idxs]
        if len(date_idxs) < 1:
            print("No valid indexes passed.")
        else:
            return date_idxs


In [69]:
headers = {'Accept': '*/*'}
params_ov = {'filtertype': 'overview', 'contracttype': 'M'}
print('Getting data from EUREX...')

Getting data from EUREX...


In [70]:
response = requests.get(url, params=params_ov, headers=headers)

if not response.ok:
    raise ValueError("Conection failed.")

In [71]:
tradingDates = pd.to_datetime(pd.Series(response.json()['header']['tradingDates']),format="%d-%m-%Y %H:%M")

In [72]:
tradingMeta = response.json()['meta']['isin']

In [73]:
print("Index     Date", tradingMeta)

Index     Date DE0008469495


In [74]:
for idx,date in enumerate(tradingDates.dt.strftime("%d-%m-%Y")):
    print(f" {idx:<6} {date:^3}")

 0      26-10-2023
 1      25-10-2023
 2      24-10-2023
 3      23-10-2023
 4      20-10-2023
 5      19-10-2023
 6      18-10-2023
 7      17-10-2023
 8      16-10-2023
 9      13-10-2023
 10     12-10-2023
 11     11-10-2023
 12     10-10-2023
 13     09-10-2023
 14     06-10-2023
 15     05-10-2023
 16     04-10-2023
 17     03-10-2023
 18     02-10-2023
 19     29-09-2023
 20     28-09-2023


In [75]:
date_idxs = get_idx(tradingDates.index)

In [76]:
params_ov['busdate'] = tradingDates[0].strftime("%Y%m%d")

In [77]:
response = requests.get(url, params=params_ov, headers=headers)
if not response.ok:
        raise ValueError("Conection failed.")

In [78]:
params_details = {}

In [79]:
if 'dataRows' in response.json(): 
    overview_df = pd.DataFrame(response.json()['dataRows'])
    overview_df = overview_df[overview_df.contractType == 'M']
    overview_df.sort_values('date',ignore_index=True,inplace=True)
    old_date_col = overview_df.loc[:,'date']
    overview_df.date = pd.to_datetime(overview_df.date,format='%Y%m%d').dt.strftime('%d-%m-%Y')
    print(overview_df)
    date_idxs2 = get_idx(overview_df.index)

          date  callVolume  callOpenInterest  putVolume  putOpenInterest  \
0   17-11-2023      7351.0           62690.0     6765.0         119621.0   
1   15-12-2023      4051.0          184419.0     9753.0         455822.0   
2   19-01-2024       787.0            2228.0     1090.0           1994.0   
3   15-03-2024      2423.0           46817.0     3597.0         156238.0   
4   21-06-2024       601.0           42926.0      466.0         107538.0   
5   20-09-2024       521.0            8165.0       18.0          17922.0   
6   20-12-2024        53.0           22327.0        6.0          28753.0   
7   21-03-2025         0.0               2.0        0.0              6.0   
8   20-06-2025         0.0            7004.0        0.0              2.0   
9   19-09-2025         0.0            1600.0        0.0              0.0   
10  19-12-2025         0.0              24.0        0.0           6329.0   
11  20-03-2026         0.0               0.0        0.0              0.0   
12  19-06-20

In [80]:
params_details['filtertype'] = 'detail'

In [81]:
params_details['busdate'] = f"{tradingDates[0].strftime('%Y%m%d')}"

In [82]:
params_details['contracttype'] = 'M'

In [83]:
params_details['productdate'] = old_date_col[0] # new contract 1 was 0

In [84]:
response = requests.get(url, params=params_details, headers=headers)
if not response.ok:
    raise ValueError("Conection failed.")

In [85]:
contractsCall_df = pd.concat([pd.DataFrame(response.json()['dataRowsCall'])])

In [86]:
contractsCall_df = contractsCall_df.sort_values(list(contractsCall_df.columns)[1:])

In [87]:
contractsPut_df = pd.concat([pd.DataFrame(response.json()['dataRowsPut'])])

In [88]:
contractsPut_df = contractsPut_df.sort_values(list(contractsPut_df.columns)[1:])

Vorletzter Handelstag: tradingDates[1]

In [89]:
params_details['busdate'] = f"{tradingDates[1].strftime('%Y%m%d')}"

In [90]:
response = requests.get(url, params=params_details, headers=headers)
if not response.ok:
    raise ValueError("Conection failed.")

In [91]:
contractsCall_l_df = pd.concat([pd.DataFrame(response.json()['dataRowsCall'])])
contractsCall_l_df = contractsCall_l_df.sort_values(list(contractsCall_l_df.columns)[1:])
contractsPut_l_df = pd.concat([pd.DataFrame(response.json()['dataRowsPut'])])
contractsPut_l_df = contractsPut_l_df.sort_values(list(contractsPut_l_df.columns)[1:])

Nächster Kontrakt, letzter Handelstag: old_date_col[1], tradingDates[0]

In [92]:
params_details['busdate'] = f"{tradingDates[0].strftime('%Y%m%d')}"

In [93]:
params_details['productdate'] = old_date_col[1] # new contract 2 was 1

In [94]:
response = requests.get(url, params=params_details, headers=headers)
if not response.ok:
    raise ValueError("Conection failed.")

In [95]:
contractsCall_N_df = pd.concat([pd.DataFrame(response.json()['dataRowsCall'])])
contractsCall_N_df = contractsCall_N_df.sort_values(list(contractsCall_N_df.columns)[1:])
contractsPut_N_df = pd.concat([pd.DataFrame(response.json()['dataRowsPut'])])
contractsPut_N_df = contractsPut_N_df.sort_values(list(contractsPut_N_df.columns)[1:])

Schrieben in ein Excelfile als Backup

In [96]:
if auswahl == "1":
# STOXX
    print("STOXX")
    with pd.ExcelWriter(f"STOXX_{tradingMeta}_{tradingDates[0].strftime('%Y_%m_%d')}_{overview_df['date'][0]}.xlsx") as writer:
        contractsCall_df.to_excel(writer, sheet_name=f"STOXX_Call_Front", index=False)
        contractsPut_df.to_excel(writer, sheet_name=f"STOXX_Put_Front", index=False)
        contractsCall_l_df.to_excel(writer, sheet_name=f"STOXX_CallFront-1", index=False)
        contractsPut_l_df.to_excel(writer, sheet_name=f"STOXX_PutFront-1", index=False)
        contractsCall_N_df.to_excel(writer, sheet_name=f"STOXX_Call+01", index=False)
        contractsPut_N_df.to_excel(writer, sheet_name=f"STOXX_Put+01", index=False)
else:
# DAX
    print ("DAX")
    with pd.ExcelWriter(f"DAX_{tradingMeta}_{tradingDates[0].strftime('%Y_%m_%d')}_{overview_df['date'][0]}.xlsx") as writer:
        contractsCall_df.to_excel(writer, sheet_name=f"Call_Front", index=False)
        contractsPut_df.to_excel(writer, sheet_name=f"Put_Front", index=False)
        contractsCall_l_df.to_excel(writer, sheet_name=f"CallFront-1", index=False)
        contractsPut_l_df.to_excel(writer, sheet_name=f"PutFront-1", index=False)
        contractsCall_N_df.to_excel(writer, sheet_name=f"Call+01", index=False)
        contractsPut_N_df.to_excel(writer, sheet_name=f"Put+01", index=False)

DAX


In [391]:
raise
#print(f"{tradingMeta}_{tradingDates[0].strftime('%Y_%m_%d')}_{overview_df['date'][0]}.xlsx")

RuntimeError: No active exception to reraise

In [ ]:
now = datetime.now()
dt_string = now.strftime("%d-%m-%Y_%H_%M_%S")

In [ ]:
akt_datum = input (f"Datum in Form 2023-08-16: ")
#akt_datum = input (f"Datum in Form " + now.strftime("%Y-%m-%d))
app = xw.App(visible = False)
# Load Wookbook

In [ ]:
if auswahl == "1":
# STOXX
    print ("STOXX")
    oi_pfad= f"C:\\Detlef\\Tradematiker\\Research\\Optionen\\OI-Abfrage\\STOXX\\STOXX_OI-Abfrage {akt_datum} V9.00.xlsm"
    print(oi_pfad)
    wb = xw.books.open(oi_pfad)
    # define sheets
    Sheet1 = wb.sheets['STOXX_Put_Front']
    Sheet2 = wb.sheets['STOXX_Call_Front']
    Sheet3 = wb.sheets['STOXX_Put+01']
    Sheet4 = wb.sheets['STOXX_Call+01']
    Sheet5 = wb.sheets['STOXX_PutFront-1']
    Sheet6 = wb.sheets['STOXX_CallFront-1']
else:
# DAX
    print ("DAX")
    oi_pfad= f"C:\\Detlef\\Tradematiker\\Research\\Optionen\\OI-Abfrage\\T_OI-Abfrage {akt_datum} V9.00.xlsm"
    print(oi_pfad)
    wb = xw.books.open(oi_pfad)
    Sheet1 = wb.sheets['Put_Front']
    Sheet2 = wb.sheets['Call_Front']
    Sheet3 = wb.sheets['Put+01']
    Sheet4 = wb.sheets['Call+01']
    Sheet5 = wb.sheets['PutFront-1']
    Sheet6 = wb.sheets['CallFront-1']

    print(oi_pfad)

#    wb = xw.Book(rf"C:\\Detlef\\Tradematiker\\Research\\Optionen\\OI-Abfrage\\T_OI-Abfrage {tradingDates[0].strftime('%Y-%m-%d')} V9.00.xlsm")


In [ ]:
#Write data into Workbook 
Sheet1.range("A1:R250").clear_contents()
Sheet1.range("A1").value ="strike"
Sheet1.range("B1").value = dt_string
Sheet1.range("J1").value ="OpenInterest"
Sheet1.range('A2').options(index=False, header=False ).value = contractsPut_df["strike"]
Sheet1.range('J2').options(index=False, header=False ).value = contractsPut_df["openInterest"]

Sheet2.range("A1:R250").clear_contents()
Sheet2.range("A1").value ="strike"
Sheet2.range("B1").value = dt_string
Sheet2.range("J1").value ="OpenInterest"
Sheet2.range('A2').options(index=False, header=False ).value = contractsCall_df["strike"]
Sheet2.range('J2').options(index=False, header=False ).value = contractsCall_df["openInterest"]
#Write data into Workbook  P/C +1
Sheet3.range("A1:R250").clear_contents()
Sheet3.range("A1").value ="strike"
Sheet3.range("B1").value = dt_string
Sheet3.range("J1").value ="OpenInterest"
Sheet3.range('A2').options(index=False, header=False ).value = contractsPut_N_df["strike"]
Sheet3.range('J2').options(index=False, header=False ).value = contractsPut_N_df["openInterest"]

Sheet4.range("A1:R250").clear_contents()
Sheet4.range("A1").value ="strike"
Sheet4.range("B1").value = dt_string
Sheet4.range("J1").value ="OpenInterest"
Sheet4.range('A2').options(index=False, header=False ).value = contractsCall_N_df["strike"]
Sheet4.range('J2').options(index=False, header=False ).value = contractsCall_N_df["openInterest"]
#Write data into Workbook 
Sheet5.range("A1:R250").clear_contents()
Sheet5.range("A1").value ="strike"
Sheet5.range("B1").value = dt_string
Sheet5.range("J1").value ="OpenInterest"
Sheet5.range('A2').options(index=False, header=False ).value = contractsPut_l_df["strike"]
Sheet5.range('J2').options(index=False, header=False ).value = contractsPut_l_df["openInterest"]

Sheet6.range("A1:R250").clear_contents()
Sheet6.range("A1").value ="strike"
Sheet6.range("B1").value = dt_string
Sheet6.range("J1").value ="OpenInterest"
Sheet6.range('A2').options(index=False, header=False ).value = contractsCall_l_df["strike"]
Sheet6.range('J2').options(index=False, header=False ).value = contractsCall_l_df["openInterest"]




In [ ]:
wb.save()
#wb.close()